In [68]:
! pip install python-dotenv

In [69]:
! pip install psycopg2

In [70]:
! pip install yelpapi

In [71]:
# Import dependencies

import pandas as pd
from pymongo import MongoClient
import pymongo

from yelpapi import YelpAPI
import requests
from config import yelp_key
from pprint import pprint
import pandas as pd
import json


In [72]:

# create a yelp api instance
yelp_api = YelpAPI(yelp_key)

print(yelp_api)

### Store CSV into DataFrame

In [13]:
zip_zhvi_single_family_homes_time_series_file = "./Resources/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_single_family_homes_time_series_df = pd.read_csv(zip_zhvi_single_family_homes_time_series_file)

# Add in Home-Type Column
zip_zhvi_single_family_homes_time_series_df["Home Type"] = "Single Family Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_single_family_homes_time_series_dict = zip_zhvi_single_family_homes_time_series_df.to_dict("records")

# Visualize
print(len(zip_zhvi_single_family_homes_time_series_df))

zip_zhvi_single_family_homes_time_series_df.head()

Remove Region ID
Sizerank
StateName
Rename region name to zip code

30342


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,960802.0,962396.0,963537.0,960187.0,955918.0,949634.0,948112.0,948911.0,954998.0,Single Family Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,298619.0,...,793957.0,795186.0,796847.0,797731.0,798012.0,798051.0,799653.0,801950.0,806333.0,Single Family Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1423693.0,1424519.0,1420952.0,1414127.0,1407642.0,1407057.0,1404420.0,1402979.0,1402128.0,Single Family Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,204122.0,...,342174.0,342397.0,342883.0,343979.0,344922.0,345504.0,345781.0,346558.0,347253.0,Single Family Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,456771.0,...,999819.0,1001592.0,1003223.0,1004383.0,1004370.0,1005620.0,1008258.0,1011170.0,1016459.0,Single Family Home


In [15]:
zip_zhvi_condo_coop_time_series_file = "./Resources/Zip_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_condo_coop_time_series_df = pd.read_csv(zip_zhvi_condo_coop_time_series_file)

# Add in Home-Type Column
zip_zhvi_condo_coop_time_series_df["Home Type"] = "Condos and Co-op Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_condo_coop_time_series_dict = zip_zhvi_condo_coop_time_series_df.to_dict("records")

# Visualize
print(len(zip_zhvi_condo_coop_time_series_df))
zip_zhvi_condo_coop_time_series_df.head()


10664


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,187153.0,...,997141.0,976952.0,952332.0,941857.0,934084.0,931789.0,930503.0,931641.0,935926.0,Condos and Co-op Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,194388.0,...,398247.0,399194.0,400544.0,401578.0,401754.0,402238.0,403466.0,405361.0,407901.0,Condos and Co-op Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,236945.0,...,1055892.0,1068214.0,1068853.0,1078253.0,1083853.0,1089299.0,1092674.0,1100314.0,1117604.0,Condos and Co-op Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,NaN,...,247259.0,246484.0,245794.0,245984.0,246056.0,246320.0,246270.0,246636.0,246470.0,Condos and Co-op Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,202096.0,...,459621.0,460100.0,461202.0,461424.0,461303.0,461430.0,462691.0,464602.0,467196.0,Condos and Co-op Home


In [16]:

zip_zhvi_one_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_one_bedroom_time_series_df = pd.read_csv(zip_zhvi_one_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_one_bedroom_time_series_df["Home Type"] = "One Bedroom Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_one_bedroom_time_series_dict = zip_zhvi_one_bedroom_time_series_df.to_dict("records")

# Visualize
print(len(zip_zhvi_one_bedroom_time_series_df))
zip_zhvi_one_bedroom_time_series_df.head()


18560


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,102766.0,...,657379.0,657478.0,658265.0,657925.0,656708.0,655825.0,654157.0,651796.0,648394.0,One Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,117621.0,...,234361.0,234935.0,235555.0,236073.0,236282.0,236818.0,237992.0,239587.0,241320.0,One Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,158083.0,...,679903.0,678316.0,677287.0,675021.0,673560.0,673517.0,672950.0,672538.0,672618.0,One Bedroom Home
3,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,133907.0,...,265891.0,265954.0,266367.0,266759.0,267050.0,267434.0,267815.0,267932.0,267699.0,One Bedroom Home
4,91940,5,77449,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,79426.0,...,126198.0,126739.0,127296.0,127984.0,128896.0,129507.0,130055.0,130456.0,131683.0,One Bedroom Home


In [17]:

zip_zhvi_two_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_two_bedroom_time_series_df = pd.read_csv(zip_zhvi_two_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_two_bedroom_time_series_df["Home Type"] = "Two Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_two_bedroom_time_series_dict = zip_zhvi_two_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_two_bedroom_time_series_df))
zip_zhvi_two_bedroom_time_series_df.head()


26681


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,211516.0,...,1115816.0,1115066.0,1112470.0,1110355.0,1106770.0,1104648.0,1103091.0,1101730.0,1100391.0,Two Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,199947.0,...,411315.0,412108.0,413346.0,414344.0,414549.0,415063.0,416473.0,418371.0,420872.0,Two Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1721495.0,1718819.0,1715486.0,1711585.0,1707480.0,1705946.0,1701921.0,1698954.0,1699372.0,Two Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,NaN,...,269660.0,269406.0,269428.0,270495.0,271643.0,272388.0,272513.0,273061.0,273637.0,Two Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,217292.0,...,467846.0,468056.0,468974.0,469130.0,469143.0,469344.0,470710.0,472715.0,475446.0,Two Bedroom Home


In [18]:

zip_zhvi_three_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_three_bedroom_time_series_df = pd.read_csv(zip_zhvi_three_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_three_bedroom_time_series_df["Home Type"] = "Three Bedroom Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_three_bedroom_time_series_dict = zip_zhvi_three_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_three_bedroom_time_series_df))
zip_zhvi_three_bedroom_time_series_df.head()


28728


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,992309.0,...,1929377.0,1919420.0,1910535.0,1904880.0,1901163.0,1901511.0,1902212.0,1907083.0,1920389.0,Three Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,249507.0,...,579817.0,581079.0,583177.0,584626.0,585485.0,586645.0,588865.0,592509.0,597737.0,Three Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2672494.0,2663477.0,2661284.0,2646807.0,2638973.0,2628182.0,2605343.0,2573719.0,2542950.0,Three Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,144763.0,...,266793.0,266797.0,267130.0,268143.0,269206.0,269880.0,270129.0,270714.0,271267.0,Three Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,313324.0,...,687148.0,687108.0,687736.0,687137.0,686859.0,686394.0,687508.0,689081.0,693130.0,Three Bedroom Home


In [19]:

zip_zhvi_four_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_four_bedroom_time_series_df = pd.read_csv(zip_zhvi_four_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_four_bedroom_time_series_df["Home Type"] = "Four Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_four_bedroom_time_series_dict = zip_zhvi_four_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_four_bedroom_time_series_df))
zip_zhvi_four_bedroom_time_series_df.head()


26467


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2408740.0,2409014.0,2404003.0,2391342.0,2388913.0,2398323.0,2411226.0,2428734.0,2455802.0,Four Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,366999.0,...,921170.0,921419.0,922116.0,922445.0,922196.0,922128.0,924016.0,927506.0,933594.0,Four Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,5955760.0,5902502.0,5836962.0,5796591.0,5736771.0,5705849.0,5656754.0,5598082.0,5511871.0,Four Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,195492.0,...,343105.0,343293.0,343752.0,344796.0,345678.0,346212.0,346470.0,347230.0,347967.0,Four Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,471019.0,...,1134482.0,1137006.0,1139337.0,1140929.0,1140258.0,1141648.0,1145064.0,1148874.0,1155107.0,Four Bedroom Home


In [20]:
# 5 and 5 plus bedroom homes
zip_zhvi_five_and_five_plus_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_5_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_five_and_five_plus_bedroom_time_series_df = pd.read_csv(zip_zhvi_five_and_five_plus_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_five_and_five_plus_bedroom_time_series_df["Home Type"] = "Five and Five Plus Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_five_and_five_plus_bedroom_time_series_dict = zip_zhvi_five_and_five_plus_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_five_and_five_plus_bedroom_time_series_df))
zip_zhvi_five_and_five_plus_bedroom_time_series_df.head()


21871


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,3526803.0,3588803.0,3644455.0,3658205.0,3682981.0,3758156.0,3896928.0,4024298.0,4116546.0,Five and Five Plus Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,NaN,...,1448109.0,1450282.0,1453442.0,1455531.0,1457204.0,1459856.0,1465304.0,1471079.0,1480683.0,Five and Five Plus Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,8831883.0,8756920.0,8598371.0,8426361.0,8327894.0,8333295.0,8323928.0,8329391.0,8286910.0,Five and Five Plus Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,240085.0,...,447991.0,448243.0,449130.0,450705.0,452194.0,453019.0,453588.0,454693.0,455474.0,Five and Five Plus Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,699792.0,...,1833778.0,1837304.0,1839939.0,1842239.0,1843137.0,1848439.0,1857536.0,1866711.0,1881082.0,Five and Five Plus Bedroom Home


### Connect to local database

In [73]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and homes collection
db = client.data_etl_app
homes_collection = db.homes

In [74]:
# Load the collection
homes_collection.delete_many({})

#data.reset_index(inplace=True)
#data_dict = data.to_dict("records")
# Insert collection
homes_collection.insert_many(zip_zhvi_single_family_homes_time_series_dict)
homes_collection.insert_many(zip_zhvi_condo_coop_time_series_dict)
homes_collection.insert_many(zip_zhvi_one_bedroom_time_series_dict)
homes_collection.insert_many(zip_zhvi_two_bedroom_time_series_dict)
homes_collection.insert_many(zip_zhvi_three_bedroom_time_series_dict)
homes_collection.insert_many(zip_zhvi_four_bedroom_time_series_dict)
homes_collection.insert_many(zip_zhvi_five_and_five_plus_bedroom_time_series_dict)

### Create new data with select columns

In [78]:
# Fill in parameters for term, location and search_limit
#term = 'Grocery'
location = 'San Francisco, CA'
#search_limit = 50
#category = 'Grocery (grocery)'
# Create the response
response = yelp_api.search_query(location = location)

# Check data type of response
type(response)

dict

In [79]:
pprint(response)

{'businesses': [{'alias': 'bi-rite-creamery-san-francisco',
                 'categories': [{'alias': 'icecream',
                                 'title': 'Ice Cream & Frozen Yogurt'}],
                 'coordinates': {'latitude': 37.761591,
                                 'longitude': -122.425717},
                 'display_phone': '(415) 626-5600',
                 'distance': 946.3867385272524,
                 'id': 'wGl_DyNxSv8KUtYgiuLhmA',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/iPNJKlOQ7-eyqa4Yv2r2BA/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '3692 18th St',
                              'address2': None,
                              'address3': '',
                              'city': 'San Francisco',
                              'country': 'US',
                              'display_address': ['3692 18th St',
                                                  'San Francisco, CA 94110'],
           

In [80]:
# Check key of response dictionary (Alternative to scroll through text to look for keys)
print(response.keys())

dict_keys(['businesses', 'total', 'region'])


In [93]:
# Set list of keys as column names
cols = list(response['businesses'][0].keys())

# Ceate a DataFrame
yelp_df = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    yelp_df = yelp_df.append(biz, ignore_index=True)

yelp_dict = yelp_df.to_dict("records")
    
yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wGl_DyNxSv8KUtYgiuLhmA,bi-rite-creamery-san-francisco,Bi-Rite Creamery,https://s3-media2.fl.yelpcdn.com/bphoto/iPNJKl...,False,https://www.yelp.com/biz/bi-rite-creamery-san-...,9622,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.5,"{'latitude': 37.761591, 'longitude': -122.425717}",[delivery],$,"{'address1': '3692 18th St', 'address2': None,...",+14156265600,(415) 626-5600,946.386739
1,lJAGnYzku5zSaLnQ_T6_GQ,brendas-french-soul-food-san-francisco-5,Brenda's French Soul Food,https://s3-media3.fl.yelpcdn.com/bphoto/sNIJne...,False,https://www.yelp.com/biz/brendas-french-soul-f...,11055,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 37.7829016035273, 'longitude': -1...",[delivery],$$,"{'address1': '652 Polk St', 'address2': '', 'a...",+14153458100,(415) 345-8100,2885.389131
2,WavvLdfdP6g8aZTtbBQHTw,gary-danko-san-francisco,Gary Danko,https://s3-media3.fl.yelpcdn.com/bphoto/CAprIo...,False,https://www.yelp.com/biz/gary-danko-san-franci...,5527,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 37.80587, 'longitude': -122.42058}",[],$$$$,"{'address1': '800 N Point St', 'address2': '',...",+14157492060,(415) 749-2060,5191.341803
3,ri7UUYmx21AgSpRsf4-9QA,tartine-bakery-and-cafe-san-francisco,Tartine Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/nPUUXY...,False,https://www.yelp.com/biz/tartine-bakery-and-ca...,7961,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,"{'latitude': 37.76131, 'longitude': -122.42431}",[delivery],$$,"{'address1': '600 Guerrero St', 'address2': ''...",+14154872600,(415) 487-2600,1087.638933
4,Xg-FyjVKAN70LO4u4Z1ozg,hog-island-oyster-co-san-francisco,Hog Island Oyster Co,https://s3-media3.fl.yelpcdn.com/bphoto/TW9FgV...,False,https://www.yelp.com/biz/hog-island-oyster-co-...,6062,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 37.795831, 'longitude': -122.393303}",[],$$,"{'address1': '1 Ferry Bldg', 'address2': '', '...",+14153917117,(415) 391-7117,5428.156707


In [96]:
# Create ned dataframe with [id, name, url, review_count, rating, transactions and price]

yelp_new_df = yelp_df[['name', 'url', 'review_count', 'rating', 'transactions','price']]

yelp_new_dict = yelp_new_df.to_dict("records")

In [97]:
yelp_new_df.head()

,name,url,review_count,rating,transactions,price
0,Bi-Rite Creamery,https://www.yelp.com/biz/bi-rite-creamery-san-...,9622,4.5,[delivery],$
1,Brenda's French Soul Food,https://www.yelp.com/biz/brendas-french-soul-f...,11055,4.0,[delivery],$$
2,Gary Danko,https://www.yelp.com/biz/gary-danko-san-franci...,5527,4.5,[],$$$$
3,Tartine Bakery & Cafe,https://www.yelp.com/biz/tartine-bakery-and-ca...,7961,4.0,[delivery],$$
4,Hog Island Oyster Co,https://www.yelp.com/biz/hog-island-oyster-co-...,6062,4.5,[],$$


### Connect to local database

In [98]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

yelp_db = client.data_etl_app
yelp_collection = yelp_db.yelp


In [99]:
# Define Yelp collection
yelp_collection = yelp_db.yelp

In [100]:
# Load the collection
yelp_collection.delete_many({})


#data.reset_index(inplace=True)
#data_dict = data.to_dict("records")
# Insert collection
yelp_collection.insert_many(yelp_new_dict)



### Store JSON data into a DataFrame

In [4]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,id,address,longitude,latitude,us_state
0,1,043 Mockingbird Place,-86.5186,39.1682,Indiana
1,2,4 Prentice Point,-85.0707,41.0938,Indiana
2,3,46 Derek Junction,-96.7776,32.7673,Texas
3,4,11966 Old Shore Place,-94.3567,39.0350,Missouri
4,5,5 Evergreen Circle,-73.9772,40.7808,New York


### Clean DataFrame

In [5]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


In [ ]:
# Query all houses
# Here, db.students refers to the collection 'classroom '
classroom = db.houses.find()

# Iterate through each student in the collection
for house in houses:
    print(house)

### Check for tables

In [11]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)

### Use pandas to load csv converted DataFrame into database

In [8]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [9]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [10]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
